# Image Enhancement: Point Operations

In [ ]:
%matplotlib inline

import numpy as np

import matplotlib.image as img
import matplotlib.pyplot as plt

from skimage import io
from skimage import exposure
from skimage.util import img_as_float32 as img_as_float

In [ ]:
def print_imginfo(I):
    print(type(I))
    print(I.shape, I.dtype)
    print('Data range:', np.min(I), 'to', np.max(I))

In [ ]:
def show_imghist(I):
    fig, ax = plt.subplots(1, 2, figsize=(10,3))
    
    ax[0].imshow(I, cmap='gray', vmin=0.0, vmax=1.0)
    ax[0].set_axis_off()
    
    ax[1].hist(I.ravel(), lw=0, bins=256, range=(0.01,0.99));
    ax[1].set_xlim(0.0,1)
    ax[1].set_yticks([])

In [ ]:
I1 = io.imread("../../images/parrot.png", as_gray=True)
I1 = img_as_float(I1)

print_imginfo(I1)
show_imghist(I1)


## Thresholding

In [ ]:
I2 = np.where((0.2 < I1) & (I1 < 0.5), 0.0, 1.0)  # if condition true else false (0=black, 255=white)

print_imginfo(I2)
show_imghist(I2)

## Contrast/Brightness, Contrast Stretching w/Thresholding

In [ ]:
I2 = exposure.rescale_intensity(I1, in_range=(0.2,0.9), out_range=(0.0,1.0))
#I2 = exposure.rescale_intensity(I1, in_range=(0.2,0.9), out_range=(0.0,1.0))

print_imginfo(I2)
show_imghist(I2)

## Gamma, Log, Inverse Log Correction

In [ ]:
#I2 = exposure.adjust_gamma(I1, 1.0, 1.0)      # gamma, gain: gain*(I**gamma)
#I2 = exposure.adjust_log(I1, 1.0)             # gain: gain*log2(I+1)
I2 = exposure.adjust_log(I1, 1.0, inv=True)   # gain: gain*(2**I-1)

print_imginfo(I2)
show_imghist(I2)

## Sigmoid Correction

In [ ]:
I2 = exposure.adjust_sigmoid(I1, 0.5, 5.0)      # cutoff, gain: 1/(1+exp(gain*(cutoff-I)))

print_imginfo(I2)
show_imghist(I2)

## Logical Operations: Masking

In [ ]:
def circular_mask(I, radius=None):
    h, w = I.shape
    center = (int(h/2), int(w/2))
    if radius is None:
        radius = min(h-center[0], w-center[1])

    X, Y = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = np.where(dist_from_center <= radius, 1.0, 0.0)
    return mask

M = circular_mask(I1,128)
I2 = np.multiply(M, I1)

print_imginfo(I2)
show_imghist(I2)

## Arithmetic Operations: Alpha-Blending using Addition

In [ ]:
F1 = io.imread("../../images/messi.jpg", as_gray=True)
F1 = img_as_float(F1)

F2 = io.imread("../../images/ronaldo.jpg", as_gray=True)
F2 = img_as_float(F2)

fig = plt.figure(figsize=(10,8))

i = 1
for alpha in np.linspace(0,1,20):
    plt.subplot(4,5,i)
    plt.imshow((1-alpha)*F1+alpha*F2, cmap='gray')
    plt.axis('off')
    i += 1
plt.subplots_adjust(wspace=0.05, hspace=0.05)
plt.show()

## Arithmetic Operations: Motion Detection using Subtraction

In [ ]:
G1 = io.imread("../../images/goal1.png", as_gray=True)
G1 = img_as_float(G1)

G2 = io.imread("../../images/goal2.png", as_gray=True)
G2 = img_as_float(G2)

G3 = G1 - G2
G4 = np.where(0.10 < np.abs(G3), 1.0, 0.0)

fig = plt.figure(figsize=(11,6))

plt.subplot(2,2,1); plt.imshow(G1, cmap='gray'); plt.axis('off')
plt.subplot(2,2,2); plt.imshow(G2, cmap='gray'); plt.axis('off')
plt.subplot(2,2,3); plt.imshow(G3, cmap='gray'); plt.axis('off')
plt.subplot(2,2,4); plt.imshow(G4, cmap='gray'); plt.axis('off')
plt.subplots_adjust(wspace=0.05, hspace=0.05)
plt.show()